In [ ]:
keep 
atr_14  , OBV

drop

'ema_cross_up', 'macd_cross_up', 'oversold_reversal', 'overbought_reversal', 'trending_market'

In [ ]:
drop_tcn = [
    'open', 'high', 'low', 'typical_price', 'EMA_21', 'SMA_20',
    'vwap_24h', 'close_4h', 'bollinger_upper', 'bollinger_lower',
    'resistance_level', 'support_level', 'high_low', 'high_close', 'low_close',
    'true_range', 'volume_mean_20', 'MACD_line', 'MACD_signal',
    'volatility_regime', 'trending_market', 'above_sma50', 'ema7_above_ema21',
    'vol_spike_1_5x', 'near_upper_band', 'near_lower_band',
    'break_upper_band', 'break_lower_band', 'rsi_oversold',
    'above_sma20', 'macd_positive', 'volume_breakout', 'volume_breakdown',
    'bullish_scenario_1', 'bullish_scenario_2', 'bullish_scenario_3',
    'bullish_scenario_4', 'bullish_scenario_5', 'bullish_scenario_6',
    'bearish_scenario_1', 'bearish_scenario_2', 'bearish_scenario_3',
    'bearish_scenario_4', 'bearish_scenario_6','ema_cross_up', 'macd_cross_up', 'oversold_reversal', 'overbought_reversal',
]

In [21]:
"""
tcn_optuna_search.py  —  Robust TCN hyper-parameter optimisation
─────────────────────────────────────────────────────────────────
• Objective  : weighted-F1 with α = 2  (precision has double weight)
• Output     : scaler, best-params (.json), all trials (.csv), history plot
• Tested on  : TensorFlow 2.16 · tcn 3.5 · Optuna 3.x  (CPU & single GPU)
"""

# ───────────────────────── imports ──────────────────────────
import os, json, gc, warnings, optuna
from datetime import datetime
from pathlib import Path
from typing import Dict, Tuple

import numpy as np
import pandas as pd
import joblib
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score

import tensorflow as tf
from tensorflow import keras
from tcn import TCN                       #  pip install tcn==3.*
from optuna_integration.tfkeras import TFKerasPruningCallback

# ───────────────────── runtime hygiene ──────────────────────
warnings.filterwarnings("ignore")
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

SEED, VAL_FRAC, ALPHA = 42, 0.20, 2.0
N_TRIALS, TIMEOUT = 40, 80 * 60  # seconds

np.random.seed(SEED)
tf.random.set_seed(SEED)
for g in tf.config.experimental.list_physical_devices("GPU"):
    tf.config.experimental.set_memory_growth(g, True)

# ──────────────────────── paths, drops ─────────────────────
CSV_PATH = Path(r"C:\Users\ADMIN\Desktop\Coding_projects\stock_market_prediction"
                r"\Stock-Market-Prediction\data\processed\gemini_btc_with_features_4h.csv")
SCALER_PKL = "tcn_scaler.pkl"

DROP_COLS = [  # (shortened – add the rest of your leakage columns here)
    "close", "open", "high", "low", "typical_price", "high_low",
    "high_close", "low_close", "EMA_21", "SMA_20", "vwap_24h", "close_4h"
]

# ───────────────────────── data load ───────────────────────
print("📊 Loading data …")
df = (
    pd.read_csv(CSV_PATH, index_col=0, parse_dates=True)
      .loc["2018-01-01":]
      .drop(columns=[c for c in DROP_COLS if c in df.columns])
      .dropna(subset=["target"])
      .dropna()
)
print(f"Shape  : {df.shape}")
print("Classes:\n", df["target"].value_counts(), "\n")

X_raw, y_raw = df.drop(columns="target").values, df["target"].astype(int).values
n_features   = X_raw.shape[1]

split = int(len(df) * (1 - VAL_FRAC))
scaler = StandardScaler().fit(X_raw[:split])
joblib.dump(scaler, SCALER_PKL)

X_tr_raw = scaler.transform(X_raw[:split]).astype(np.float32)
X_va_raw = scaler.transform(X_raw[split:]).astype(np.float32)
y_tr_raw, y_va_raw = y_raw[:split], y_raw[split:]

pos_rate = y_tr_raw.mean()
CLASS_W0 = np.float32(1.0)
CLASS_W1 = np.float32((1 - pos_rate) / pos_rate if pos_rate else 1.0)
print(f"Class-weights 0 / 1 → {CLASS_W0:.2f} / {CLASS_W1:.2f}")

# ─────────────────── window cache helper ───────────────────
_CACHE: Dict[Tuple[int, int, int], Tuple[np.ndarray, np.ndarray]] = {}

def make_windows(arr: np.ndarray, labels: np.ndarray, win: int):
    k = (id(arr), len(labels), win)
    if k in _CACHE:
        return _CACHE[k]

    Xs, ys = [], []
    for i in range(win, len(arr)):
        Xs.append(arr[i - win:i])
        ys.append(labels[i])
    Xw, yw = np.asarray(Xs, np.float32), np.asarray(ys, np.int8)

    if Xw.nbytes + yw.nbytes < 1_000_000_000:
        _CACHE[k] = (Xw, yw)
    return Xw, yw

# ─────────────── metric: weighted-F1 (α=2) ────────────────
def f1_alpha2(y_true, y_prob) -> float:
    y_pred = (y_prob >= 0.5).astype(int)
    p, r   = precision_score(y_true, y_pred, zero_division=0), \
             recall_score   (y_true, y_pred, zero_division=0)
    return 0. if p+r == 0 else (1+ALPHA)*p*r / (ALPHA*p + r)

# ──────────────── TCN model factory ───────────────────────
def build_tcn(cfg: dict) -> keras.Model:
    dilations = [2 ** i for i in range(cfg["nb_stacks"] * cfg["blocks_per_stack"])]

    inp = keras.layers.Input(shape=(cfg["window"], n_features))
    x   = TCN(
            nb_filters       = cfg["filters"],
            kernel_size      = cfg["kernel"],
            nb_stacks        = cfg["nb_stacks"],
            dilations        = dilations,
            padding          = "causal",
            dropout_rate     = cfg["dropout"],
            activation       = cfg["act"],
            use_skip_connections = True,
            use_batch_norm   = cfg["norm"] == "batch",
            use_layer_norm   = cfg["norm"] == "layer",
            return_sequences = False
         )(inp)

    x   = keras.layers.Dense(cfg["dense"], activation=cfg["act"])(x)
    x   = keras.layers.Dropout(cfg["dropout"])(x)
    out = keras.layers.Dense(1, activation="sigmoid")(x)
    model = keras.Model(inp, out)

    # balanced BCE
    def weighted_bce(y_t, y_p):
        y_t = tf.cast(y_t, y_p.dtype)
        w   = tf.where(tf.equal(y_t, 1), CLASS_W1, CLASS_W0)
        w   = tf.cast(w, y_p.dtype)
        return tf.reduce_mean(w * keras.losses.binary_crossentropy(y_t, y_p))

    model.compile(keras.optimizers.Adam(cfg["lr"]), loss=weighted_bce)
    return model

# ──────────────────── Optuna objective ────────────────────
def objective(trial: optuna.trial.Trial):
    cfg = dict(
        window           = trial.suggest_int("window", 12, 60, step=6),
        filters          = trial.suggest_categorical("filters", [32, 48, 64, 96]),
        kernel           = trial.suggest_int("kernel", 2, 6),
        nb_stacks        = trial.suggest_int("nb_stacks", 1, 3),
        blocks_per_stack = trial.suggest_int("blocks_per_stack", 1, 2),
        dropout          = trial.suggest_float("dropout", 0.05, 0.35),
        dense            = trial.suggest_categorical("dense", [32, 64, 128]),
        act              = trial.suggest_categorical("act", ["relu", "elu", "selu", tf.nn.swish]),
        lr               = trial.suggest_float("lr", 5e-5, 3e-3, log=True),
        batch            = trial.suggest_categorical("batch", [32, 64]),
        norm             = trial.suggest_categorical("norm", ["none", "batch", "layer"]),
    )

    X_tr, y_tr = make_windows(X_tr_raw, y_tr_raw, cfg["window"])
    X_va, y_va = make_windows(X_va_raw, y_va_raw, cfg["window"])

    if len(X_tr) < cfg["batch"] * 10:      # too small → prune
        return float("inf")

    tf.keras.backend.clear_session(); gc.collect()
    model = build_tcn(cfg)

    cb = [
        keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True, verbose=0),
        keras.callbacks.ReduceLROnPlateau(patience=5, factor=.5, min_lr=1e-6, verbose=0),
        TFKerasPruningCallback(trial, "val_loss")
    ]

    hist = model.fit(X_tr, y_tr,
                     validation_data=(X_va, y_va),
                     epochs=100,
                     batch_size=cfg["batch"],
                     shuffle=False,
                     callbacks=cb,
                     verbose=0)

    y_prob = model.predict(X_va, batch_size=cfg["batch"], verbose=0).ravel()
    score  = f1_alpha2(y_va, y_prob)

    # store metrics to inspect later
    trial.set_user_attr("precision", precision_score(y_va, y_prob >= 0.5, zero_division=0))
    trial.set_user_attr("recall",    recall_score   (y_va, y_prob >= 0.5, zero_division=0))
    trial.set_user_attr("best_epoch", int(np.argmin(hist.history["val_loss"]) + 1))

    # print live metrics
    print(f"Trial {trial.number:2d}  "
          f"Fα2={score:.4f}  P={trial.user_attrs['precision']:.3f} "
          f"R={trial.user_attrs['recall']:.3f}  cfg={trial.params}")

    del model; tf.keras.backend.clear_session(); gc.collect()
    return -score   # minimise

# ─────────────────────────── main ──────────────────────────
def main():
    study = optuna.create_study(direction="minimize",
                                sampler=optuna.samplers.TPESampler(seed=SEED),
                                pruner =optuna.pruners.MedianPruner(n_startup_trials=5))

    print(f"\n🚀 Optimising  {N_TRIALS} trials  |  {TIMEOUT//60} min wall-time …")
    study.optimize(objective,
                   n_trials=N_TRIALS,
                   timeout=TIMEOUT,
                   show_progress_bar=True,
                   gc_after_trial=True)

    best, ts = study.best_trial, datetime.utcnow().strftime("%Y%m%d_%H%M%S")

    print("\n═══════════ BEST RESULT ═══════════")
    print(f"weighted-F1 (α=2) : {-best.value:.4f}")
    print(f"precision         : {best.user_attrs['precision']:.4f}")
    print(f"recall            : {best.user_attrs['recall']:.4f}")
    print(f"best epoch        : {best.user_attrs['best_epoch']}")
    print("hyper-parameters  :")
    for k, v in best.params.items():
        print(f"  {k:18s}: {v}")

    json.dump(best.params, open(f"best_params_tcn_{ts}.json", "w"), indent=2)
    study.trials_dataframe().to_csv(f"trials_tcn_{ts}.csv", index=False)

    try:
        import matplotlib.pyplot as plt
        optuna.visualization.matplotlib.plot_optimization_history(study)
        plt.tight_layout()
        plt.savefig(f"optuna_tcn_history_{ts}.png", dpi=300)
        plt.close()
    except Exception:
        print("⚠️ matplotlib unavailable – history plot skipped.")

    print(f"\n📝 Artefacts saved (timestamp {ts}).  "
          f"Scaler → {SCALER_PKL}")

# ───────────────────────────────────────────────────────────
if __name__ == "__main__":
    main()


📊 Loading data …
Shape  : (15855, 59)
Classes:
 target
1    8097
0    7758
Name: count, dtype: int64 

Class-weights 0 / 1 → 1.00 / 0.97


[I 2025-06-09 18:52:10,220] A new study created in memory with name: no-name-5d78d030-0093-439e-a2ba-35523e4ee0ab



🚀 Optimising  40 trials  |  80 min wall-time …


  0%|          | 0/40 [00:00<?, ?it/s]

Trial  0  Fα2=0.1605  P=0.509 R=0.120  cfg={'window': 30, 'filters': 32, 'kernel': 2, 'nb_stacks': 1, 'blocks_per_stack': 2, 'dropout': 0.23033450352296264, 'dense': 128, 'act': 'relu', 'lr': 0.00017376356936978755, 'batch': 32, 'norm': 'batch'}


...

Trial 36  Fα2=0.7666  P=0.523 R=1.000  cfg={'window': 12, 'filters': 48, 'kernel': 5, 'nb_stacks': 3, 'blocks_per_stack': 2, 'dropout': 0.3118914406542517, 'dense': 128, 'act': 'relu', 'lr': 0.0005947328436650964, 'batch': 32, 'norm': 'layer'}


Best trial: 4. Best value: -0.765667:  90%|█████████ | 36/40 [1:21:31<04:43, 70.99s/it, 3711.39/4800 seconds]

[I 2025-06-09 20:13:41,748] Trial 36 finished with value: -0.766599597585513 and parameters: {'window': 12, 'filters': 48, 'kernel': 5, 'nb_stacks': 3, 'blocks_per_stack': 2, 'dropout': 0.3118914406542517, 'dense': 128, 'act': 'relu', 'lr': 0.0005947328436650964, 'batch': 32, 'norm': 'layer'}. Best is trial 36 with value: -0.766599597585513.


Best trial: 36. Best value: -0.7666:  92%|█████████▎| 37/40 [1:21:34<06:36, 132.27s/it, 4894.06/4800 seconds]



═══════════ BEST RESULT ═══════════
weighted-F1 (α=2) : 0.7666
precision         : 0.5226
recall            : 1.0000
best epoch        : 35
hyper-parameters  :
  window            : 12
  filters           : 48
  kernel            : 5
  nb_stacks         : 3
  blocks_per_stack  : 2
  dropout           : 0.3118914406542517
  dense             : 128
  act               : relu
  lr                : 0.0005947328436650964
  batch             : 32
  norm              : layer

📝 Artefacts saved (timestamp 20250609_171344).  Scaler → tcn_scaler.pkl


In [ ]:
[I 2025-06-09 18:05:11,711] A new study created in memory with name: no-name-0933098c-6339-4e53-ace1-0af2750e397a
📊 Loading data …
Shape  : (15855, 66)
Classes:
 target
1    8097
0    7758
Name: count, dtype: int64 

Class-weights 0 / 1 → 1.00 / 0.97

🚀 Optimising  40 trials  |  30 min wall-time …
  0%|          | 0/40 [00:00<?, ?it/s]Trial  0  Fα2=0.4618  P=0.529 R=0.434  cfg={'window': 30, 'filters': 32, 'kernel': 2, 'nb_stacks': 1, 'blocks_per_stack': 2, 'dropout': 0.23033450352296264, 'dense': 128, 'act': 'relu', 'lr': 0.00017376356936978755, 'batch': 32, 'norm': 'batch'}
  0%|          | 0/40 [00:44<?, ?it/s][I 2025-06-09 18:05:56,603] Trial 0 finished with value: -0.46183783783783783 and parameters: {'window': 30, 'filters': 32, 'kernel': 2, 'nb_stacks': 1, 'blocks_per_stack': 2, 'dropout': 0.23033450352296264, 'dense': 128, 'act': 'relu', 'lr': 0.00017376356936978755, 'batch': 32, 'norm': 'batch'}. Best is trial 0 with value: -0.46183783783783783.
Best trial: 0. Best value: -0.461838:   2%|▎         | 1/40 [00:45<29:26, 45.29s/it, 45.29/1800 seconds]Trial  1  Fα2=0.2813  P=0.515 R=0.229  cfg={'window': 24, 'filters': 64, 'kernel': 4, 'nb_stacks': 2, 'blocks_per_stack': 1, 'dropout': 0.2322634555704315, 'dense': 128, 'act': 'relu', 'lr': 0.0008234548958371457, 'batch': 32, 'norm': 'layer'}
Best trial: 0. Best value: -0.461838:   2%|▎         | 1/40 [02:14<29:26, 45.29s/it, 45.29/1800 seconds][I 2025-06-09 18:07:26,384] Trial 1 finished with value: -0.28134328358208954 and parameters: {'window': 24, 'filters': 64, 'kernel': 4, 'nb_stacks': 2, 'blocks_per_stack': 1, 'dropout': 0.2322634555704315, 'dense': 128, 'act': 'relu', 'lr': 0.0008234548958371457, 'batch': 32, 'norm': 'layer'}. Best is trial 0 with value: -0.46183783783783783.
Best trial: 0. Best value: -0.461838:   5%|▌         | 2/40 [02:15<45:16, 71.48s/it, 135.10/1800 seconds]Trial  2  Fα2=0.5694  P=0.519 R=0.599  cfg={'window': 24, 'filters': 32, 'kernel': 2, 'nb_stacks': 3, 'blocks_per_stack': 2, 'dropout': 0.3318496824692567, 'dense': 128, 'act': <function swish at 0x0000025F5ECDAF80>, 'lr': 0.0002455257311459749, 'batch': 64, 'norm': 'layer'}
Best trial: 0. Best value: -0.461838:   5%|▌         | 2/40 [13:03<45:16, 71.48s/it, 135.10/1800 seconds][I 2025-06-09 18:18:15,273] Trial 2 finished with value: -0.5694444444444443 and parameters: {'window': 24, 'filters': 32, 'kernel': 2, 'nb_stacks': 3, 'blocks_per_stack': 2, 'dropout': 0.3318496824692567, 'dense': 128, 'act': <function swish at 0x0000025F5ECDAF80>, 'lr': 0.0002455257311459749, 'batch': 64, 'norm': 'layer'}. Best is trial 2 with value: -0.5694444444444443.
Best trial: 2. Best value: -0.569444:   8%|▊         | 3/40 [13:04<3:26:43, 335.23s/it, 784.19/1800 seconds]Trial  3  Fα2=0.4582  P=0.528 R=0.430  cfg={'window': 18, 'filters': 64, 'kernel': 2, 'nb_stacks': 1, 'blocks_per_stack': 2, 'dropout': 0.26205720315428516, 'dense': 64, 'act': 'selu', 'lr': 0.00019380132456492117, 'batch': 64, 'norm': 'batch'}
Best trial: 2. Best value: -0.569444:   8%|▊         | 3/40 [13:50<3:26:43, 335.23s/it, 784.19/1800 seconds][I 2025-06-09 18:19:01,809] Trial 3 finished with value: -0.45815358067299394 and parameters: {'window': 18, 'filters': 64, 'kernel': 2, 'nb_stacks': 1, 'blocks_per_stack': 2, 'dropout': 0.26205720315428516, 'dense': 64, 'act': 'selu', 'lr': 0.00019380132456492117, 'batch': 64, 'norm': 'batch'}. Best is trial 2 with value: -0.5694444444444443.
Best trial: 2. Best value: -0.569444:  10%|█         | 4/40 [13:50<2:12:44, 221.23s/it, 830.66/1800 seconds]Trial  4  Fα2=0.0734  P=0.464 R=0.052  cfg={'window': 54, 'filters': 96, 'kernel': 4, 'nb_stacks': 3, 'blocks_per_stack': 1, 'dropout': 0.2068198488145982, 'dense': 32, 'act': 'elu', 'lr': 0.0020547569815878254, 'batch': 64, 'norm': 'none'}
Best trial: 2. Best value: -0.569444:  10%|█         | 4/40 [20:56<2:12:44, 221.23s/it, 830.66/1800 seconds][I 2025-06-09 18:26:07,757] Trial 4 finished with value: -0.07344797435150102 and parameters: {'window': 54, 'filters': 96, 'kernel': 4, 'nb_stacks': 3, 'blocks_per_stack': 1, 'dropout': 0.2068198488145982, 'dense': 32, 'act': 'elu', 'lr': 0.0020547569815878254, 'batch': 64, 'norm': 'none'}. Best is trial 2 with value: -0.5694444444444443.
Best trial: 2. Best value: -0.569444:  12%|█▎        | 5/40 [21:44<2:52:07, 295.06s/it, 1256.64/1800 seconds][I 2025-06-09 18:26:56,640] Trial 5 pruned. Trial was pruned at epoch 2.
Best trial: 2. Best value: -0.569444:  15%|█▌        | 6/40 [22:07<1:59:46, 211.37s/it, 1305.56/1800 seconds][I 2025-06-09 18:27:19,541] Trial 6 pruned. Trial was pruned at epoch 0.
Best trial: 2. Best value: -0.569444:  18%|█▊        | 7/40 [22:51<1:22:22, 149.77s/it, 1328.51/1800 seconds][I 2025-06-09 18:28:03,649] Trial 7 pruned. Trial was pruned at epoch 0.
Best trial: 2. Best value: -0.569444:  20%|██        | 8/40 [25:29<1:01:56, 116.16s/it, 1372.68/1800 seconds][I 2025-06-09 18:30:41,459] Trial 8 pruned. Trial was pruned at epoch 2.
Best trial: 2. Best value: -0.569444:  22%|██▎       | 9/40 [25:45<1:06:46, 129.23s/it, 1530.66/1800 seconds][I 2025-06-09 18:30:57,079] Trial 9 pruned. Trial was pruned at epoch 0.
Best trial: 2. Best value: -0.569444:  25%|██▌       | 10/40 [26:39<47:04, 94.15s/it, 1546.25/1800 seconds]  [I 2025-06-09 18:31:51,377] Trial 10 pruned. Trial was pruned at epoch 0.
Best trial: 2. Best value: -0.569444:  28%|██▊       | 11/40 [26:48<39:37, 82.00s/it, 1600.71/1800 seconds][I 2025-06-09 18:32:00,674] Trial 11 pruned. Trial was pruned at epoch 0.
Best trial: 2. Best value: -0.569444:  30%|███       | 12/40 [26:57<27:56, 59.86s/it, 1609.93/1800 seconds][I 2025-06-09 18:32:09,118] Trial 12 pruned. Trial was pruned at epoch 0.
Best trial: 2. Best value: -0.569444:  32%|███▎      | 13/40 [27:06<19:56, 44.30s/it, 1618.42/1800 seconds][I 2025-06-09 18:32:18,590] Trial 13 pruned. Trial was pruned at epoch 0.
Best trial: 2. Best value: -0.569444:  35%|███▌      | 14/40 [27:38<14:38, 33.78s/it, 1627.88/1800 seconds][I 2025-06-09 18:32:50,659] Trial 14 pruned. Trial was pruned at epoch 0.
Best trial: 2. Best value: -0.569444:  38%|███▊      | 15/40 [27:39<13:51, 33.27s/it, 1659.99/1800 seconds]Trial 15  Fα2=0.5717  P=0.532 R=0.594  cfg={'window': 30, 'filters': 32, 'kernel': 3, 'nb_stacks': 1, 'blocks_per_stack': 2, 'dropout': 0.19135781227800264, 'dense': 32, 'act': 'relu', 'lr': 5.35786547006468e-05, 'batch': 32, 'norm': 'layer'}
Best trial: 2. Best value: -0.569444:  38%|███▊      | 15/40 [28:35<13:51, 33.27s/it, 1659.99/1800 seconds][I 2025-06-09 18:33:47,413] Trial 15 finished with value: -0.5716803760282021 and parameters: {'window': 30, 'filters': 32, 'kernel': 3, 'nb_stacks': 1, 'blocks_per_stack': 2, 'dropout': 0.19135781227800264, 'dense': 32, 'act': 'relu', 'lr': 5.35786547006468e-05, 'batch': 32, 'norm': 'layer'}. Best is trial 15 with value: -0.5716803760282021.
Best trial: 15. Best value: -0.57168:  40%|████      | 16/40 [29:14<16:08, 40.33s/it, 1716.72/1800 seconds][I 2025-06-09 18:34:26,236] Trial 16 pruned. Trial was pruned at epoch 0.
Best trial: 15. Best value: -0.57168:  42%|████▎     | 17/40 [29:29<15:18, 39.93s/it, 1755.72/1800 seconds][I 2025-06-09 18:34:41,644] Trial 17 pruned. Trial was pruned at epoch 0.
Best trial: 15. Best value: -0.57168:  45%|████▌     | 18/40 [32:38<11:56, 32.56s/it, 1771.12/1800 seconds][I 2025-06-09 18:37:50,265] Trial 18 pruned. Trial was pruned at epoch 1.
Best trial: 15. Best value: -0.57168:  48%|████▊     | 19/40 [32:39<36:06, 103.15s/it, 1959.93/1800 seconds]

═══════════ BEST RESULT ═══════════
weighted-F1 (α=2) : 0.5717
precision         : 0.5323
recall            : 0.5937
best epoch        : 7
hyper-parameters  :
  window            : 30
  filters           : 32
  kernel            : 3
  nb_stacks         : 1
  blocks_per_stack  : 2
  dropout           : 0.19135781227800264
  dense             : 32
  act               : relu
  lr                : 5.35786547006468e-05
  batch             : 32
  norm              : layer

📝 Artefacts saved (timestamp 20250609_153751).  Scaler → tcn_scaler.pkl